# 타입B 중간보스

In [1]:
import re
from tqdm.notebook import tnrange
import csv
import json
from os import walk
import os
import pandas as pd
import datetime

In [2]:
# 해당 경로의 파일리스트 추출
def get_filelist(input_repo):
    file_list = []
    for (dirpath, dirnames, filenames) in walk(input_repo):
        file_list.extend(filenames)
        break

    return file_list

In [3]:
# 해당 경로의 csv파일의 경로+파일명 리스트 추출
def get_csvlist(input_repo, file_list):
    csv_list = []
    for i in range(len(file_list)):
        filename, file_extension = os.path.splitext(f'{input_repo}/{file_list[i]}')

        if file_extension == '.csv':
            csv_list.append(filename+file_extension)

    return csv_list

In [4]:
def concat_csvfile(csv_list):
    COLUMNS = ['lineNumber', 'originalText', 'transcription']
    result = pd.concat([pd.read_csv(f) for f in csv_list])    # 라인넘버 기준 안합친파일 합치기
    sorted_result_not_flat = result.sort_values(by=['lineNumber'])    # 작업물 sort by lineNumber
    sorted_result_not_flat.to_csv(f'{output_repo}/temp.csv', index=False, encoding='utf-8-sig')
    sorted_result = pd.read_csv(f'{output_repo}/temp.csv')
    os.remove(f'{output_repo}/temp.csv')
    
    return sorted_result

In [5]:
# 전체 문장에서 태그 안만 뽑음
def save_tag(sentence):
    regex = re.compile('(?<=\<{3}).*(?=\>{3})')
    result = regex.search(sentence)
    
    return result

In [6]:
# 전체 문장으로부터 태그 제거
def remove_tag(sentence):
    result = sentence.replace('<<<', '').replace('>>>', '')
        
    return result

In [7]:
# 태그 내 문장으로부터 법률어휘 추출
def draw_word(sentence_tagremoved):
    regex = re.compile('(?<=\<=).*(?=\>)')
    result = regex.search(sentence_tagremoved)
    
    try:
        return result.group()
    except:
        return 'no word'

In [8]:
# csv형태로 저장
def save_as_csv(input_repo, output_repo, result):
    year = str(datetime.datetime.now().year)[2:]
    month = str(datetime.datetime.now().month)
    day = str(datetime.datetime.now().day)
    hour = str(datetime.datetime.now().hour)
    minute = str(datetime.datetime.now().minute)

    print(f'{output_repo}의 경로에 태그안뽑기_{year}{month}{day}_{hour}{minute}.csv의 이름으로 파일 생성이 완료되었습니다.')

    result.to_csv(f'{output_repo}/태그안뽑기_{year}{month}{day}_{hour}{minute}.csv', index=False, encoding='utf-8-sig')

In [9]:
input_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/concat'
ref_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/accum_concat/법률어휘 살리기 최종'
output_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기'

ref_filename = 'total_with_lineNumber.csv'

In [10]:
file_list = get_filelist(input_repo)
csv_list = get_csvlist(input_repo, file_list)
sorted_result = concat_csvfile(csv_list)
ref_word = pd.read_csv(f'{ref_repo}/{ref_filename}')

In [11]:
result_dataframe = pd.DataFrame(columns=['lineNumber', 'word', 'originalText', 'typeA', 'transcription', 'tag_removed'])
for i in tnrange(len(sorted_result)):
    lineNumber = sorted_result['lineNumber'][i]
    word = list(ref_word[ref_word['lineNumber']==sorted_result['lineNumber'][i]]['word'])[0]
    originalText = list(ref_word[ref_word['lineNumber']==sorted_result['lineNumber'][i]]['org'])[0]
    typeA = sorted_result['originalText'][i]
    transcription = sorted_result['transcription'][i]
    tag_removed = re.sub('(\<=.*?\>{1})','',str(sorted_result['transcription'][i])).replace('<<<', '').replace('>>>', '')
    result_dataframe = result_dataframe.append({'lineNumber':lineNumber, 'word':word, 'originalText':originalText, 'typeA':typeA, 'transcription':transcription, 'tag_removed':tag_removed}, ignore_index=True)

  0%|          | 0/76870 [00:00<?, ?it/s]

In [12]:
save_as_csv(input_repo, output_repo, result_dataframe)

/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기의 경로에 태그안뽑기_211115_198.csv의 이름으로 파일 생성이 완료되었습니다.
